# Analyse pour le plan de maintenance - segmentation client Olist

-------------------------------------------------------------------------------------------------------------------------------

Pour rappel, nous avons crée une segmentation client pour Olist.
Nous disposons de diverses tables concernant les clients, les commandes...

Nous avons choisi d'utiliser un modèle Kmeans (5 clusters). Il nous faut à présent définir la fréquence de mise à jour du modèle. Le but étant d'avoir une segmentation qui correspond aux clients.

Importons les bibliothèques.

In [3]:
#notre package de fonctionnalités
from Package import Scripts_Analyse01 as pk
from Package_Olist import Scripts_Olist as pk_olist

import pandas as pd
import numpy as np
import missingno
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy.stats
from scipy.stats import pearsonr
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from yellowbrick.features import ParallelCoordinates
from plotly.graph_objects import Layout
import jenkspy
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.features import PCA as PCA_yellow
from yellowbrick.style import set_palette
# Import train_test_split function
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectKBest, f_classif,mutual_info_classif
from skfeature.function.similarity_based import fisher_score
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
 
import mlflow
from datetime import datetime, timedelta

from sklearn import preprocessing
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.cluster import AgglomerativeClustering

Nous réutilisons nos fonctions, insérer dans le package OLIST pour importer les données.

In [7]:
with pk_olist.timer("Full complete run"):
        df = pk_olist.main(False)

df shape: 99441
Order df shape: (99441, 17)
Orders  - done in 1s
df shape: 99441
reviews shape: 99992
reviews agg shape: 99441
Reviews df shape: (99441, 5)
df reviews (99441, 22)
Orders reviews - done in 23s
Order payments df shape: (99440, 14)
df payment (99441, 35)
Process order payments - done in 1s
Order items df shape: (98666, 82)
Process order items - done in 1s
orders informations shape: (99441, 117)
Orders informations - done in 26s
Customer df shape: (99441, 121)
Process customer - done in 2s
Full complete run - done in 28s


Appliquons les mêmes modifications et créons les variables RFM.

In [8]:
pk.data_uniqueone_string(df)
for i in df.columns:
    if "product_" in i:
        df[i]=df[i].fillna(0)

for i in df.columns:
    if ("payment_" in i) or ("price_" in i) or ("freight_" in i)  or ("review_" in i):
        df[i]=df[i].fillna(0)

for i in df.columns:
    if ("time_" in i):
        df[i]=df[i].fillna(max(df[i]))

del df["customer_id_size"]

#build 4 clusters for recency and add it to dataframe
kmeans = KMeans(n_clusters=5)
kmeans.fit(df[['order_purchase_timestamp_recency']])
df['RecencyCluster'] = kmeans.predict(df.loc[:,['order_purchase_timestamp_recency']])

#function for ordering cluster numbers
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

df_new = order_cluster('RecencyCluster', 'order_purchase_timestamp_recency',df,False)




customer_unique_id - count unique : 96096
customer_id_size - count unique : 9
order_id_size - count unique : 9
order_purchase_timestamp_max - count unique : 95834
time_delivered_approved_customer_mean - count unique : 251
time_approval_purchase_mean - count unique : 37
time_delivery_carrier_approved_mean - count unique : 128
time_delivery_customer_carrier_mean - count unique : 248
time_delivery_customer_estimated_mean - count unique : 331
order_status_approved_sum - count unique : 2
order_status_canceled_sum - count unique : 4
order_status_created_sum - count unique : 2
order_status_delivered_sum - count unique : 10
order_status_invoiced_sum - count unique : 2
order_status_processing_sum - count unique : 2
order_status_shipped_sum - count unique : 3
order_status_unavailable_sum - count unique : 3
deleted unique colonne : order_status_nan_sum
review_score_mean_mean - count unique : 34
review_comment_message_ind_sum_mean - count unique : 17
review_comment_title_ind_sum_mean - count uniqu

In [9]:
#k-means
kmeans = KMeans(n_clusters=3)
kmeans.fit(df_new[['order_id_size']])
df_new['FrequencyCluster'] = kmeans.predict(df_new[['order_id_size']])

#order the frequency cluster
df_new = order_cluster('FrequencyCluster', 'order_id_size',df_new,True)

#see details of each cluster
df_new.groupby('FrequencyCluster')['order_id_size'].describe()

,count,mean,std,min,25%,50%,75%,max
FrequencyCluster,,,,,,,,
0,93099.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
1,2745.0,2.000000,0.000000,2.0,2.0,2.0,2.0,2.0
2,252.0,3.380952,1.196656,3.0,3.0,3.0,3.0,17.0


In [10]:
#apply clustering
kmeans = KMeans(init='k-means++',n_clusters=5, random_state=0, max_iter=500)
kmeans.fit(df_new[['payment_value_sum_sum']])
df_new['RevenueCluster'] = kmeans.predict(df_new[['payment_value_sum_sum']])


#order the cluster numbers
df_new = order_cluster('RevenueCluster', 'payment_value_sum_sum',df_new,True)

#show details of the dataframe
df_new.groupby('RevenueCluster')['payment_value_sum_sum'].describe()

,count,mean,std,min,25%,50%,75%,max
RevenueCluster,,,,,,,,
0,72704.0,90.459855,43.908957,0.00,53.9600,83.57,123.9700,186.77
1,19219.0,281.977185,83.691731,186.79,214.0050,255.56,333.3550,521.64
2,3451.0,759.399687,191.781011,522.08,607.8700,706.33,872.6900,1274.51
3,692.0,1788.040405,463.266883,1275.39,1422.9200,1653.60,2031.6025,3406.47
4,30.0,5158.233667,2196.626544,3526.46,3864.9875,4185.01,5763.5150,13664.08


Il convient maintenant de créer un pipeline qui va nous permettre de calculer le ARI sur une période choisie.

## Création du pipeline

L'indice de Rand index ajusté calcule une mesure de similarité entre deux regroupements en considérant toutes les paires d'échantillons et en comptant les paires qui sont attribuées dans le même groupe ou dans des groupes différents dans les regroupements prédits et ceux définis comme vrais.

Nous allons donc calculer le ARI entre : 
- les classes prédites par le modèle de départ sur les données à t+30jours 
- et les classes obtenues par le nouveau modèle entrainé sur les données t+30jours
Cela nous permettra de voir à quel moment notre ARI baisse et, s'il faut le mettre à jour.

Créons notre pipeline.

In [64]:
metrics_history=[]

#order_purchase_timestamp_max
def model_analyse(date_1, n_days, file, var_time):
    #entrainement m0
    file0=file.loc[file[var_time]<=date_1]
    file0_filter = file0.loc[:, ["RevenueCluster", "FrequencyCluster", "RecencyCluster"]]
    
    #standardise
    std_scale0 = preprocessing.StandardScaler().fit(file0_filter)
    file0_std0 = std_scale0.transform(file0_filter)
    
    #entrainement du m0
    kmeans_m0 = KMeans(init='k-means++',n_clusters=5, random_state=0, max_iter=500)
    kmeans_m0.fit(file0_std0)
    C0 = kmeans_m0.labels_
    
    #file 1
    end_date = pd.to_datetime(date_1) + timedelta(days=n_days)
    file1=file.loc[(file[var_time]>date_1) & (file[var_time]<end_date)]
    file1_filter = file1.loc[:, ["RevenueCluster", "FrequencyCluster", "RecencyCluster"]]
    
    #entrainement m1
    kmeans_m1 = KMeans(init='k-means++',n_clusters=5, random_state=0, max_iter=500)
    std_scale1 = preprocessing.StandardScaler().fit(file1_filter)
    file1_std1 = std_scale1.transform(file1_filter)
    if file1_std1.shape[0]>=5:
        kmeans_m1.fit(file1_std1)
        C1_new = kmeans_m1.labels_
        print("------------c1 new:",C1_new)
        #predict file 1 avec m0
        file1_std0 = std_scale0.transform(file1_filter)
        C1_init = kmeans_m0.predict(file1_std0)

        print("------------c1 init:",C1_init)
        #calcul ARI
        if (len(C1_init)>0) and (len(C1_new)>0):
            ARI=metrics.adjusted_rand_score(C1_init, C1_new)
        else:
            ARI=0
    else:
        ARI=0
    return ARI
    

def find_best_periode_update_model(date1, n_days, file, var_time):
    num_model=1
    date_end=max(file[var_time])
    date1=pd.to_datetime(date1)
    while date1<=date_end:
        ARI = model_analyse(date1, n_days, file, var_time)
        
        date0 = pd.to_datetime(date1)
        date1 = pd.to_datetime(date1) + timedelta(days=n_days)
        print('model: '+str(num_model), ' - n_days: '+str(n_days), ' - date1: '+str(date1),' - date0: '+str(date0), ' - ARI: '+str(ARI))
        metrics_history.append({'model':num_model, 'n_days':n_days, 'date1':date1,'date0':date0, 'ARI':ARI})
        num_model=num_model+1

In [12]:
min(df_new["order_purchase_timestamp_max"])

Timestamp('2018-10-17 17:30:18')

In [16]:
pd.to_datetime("2017-12-31")

Timestamp('2017-12-31 00:00:00')

In [59]:
file=df_new.copy()
del file["customer_unique_id"]

## Lancement du pipeline

In [65]:
find_best_periode_update_model("2017-12-31", 15, file, "order_purchase_timestamp_max")

------------c1 new: [1 1 1 ... 2 2 2]
------------c1 init: [0 0 0 ... 3 3 3]
model: 1  - n_days: 15  - date1: 2018-01-15 00:00:00  - date0: 2017-12-31 00:00:00  - ARI: 0.9772097423721141
------------c1 new: [0 0 0 ... 2 4 4]
------------c1 init: [0 0 0 ... 3 3 3]
model: 2  - n_days: 15  - date1: 2018-01-30 00:00:00  - date0: 2018-01-15 00:00:00  - ARI: 0.9803183683168587
------------c1 new: [0 0 0 ... 3 3 3]
------------c1 init: [0 0 0 ... 3 3 3]
model: 3  - n_days: 15  - date1: 2018-02-14 00:00:00  - date0: 2018-01-30 00:00:00  - ARI: 0.23171446521651934
------------c1 new: [0 0 0 ... 4 4 4]
------------c1 init: [3 3 3 ... 2 2 2]
model: 4  - n_days: 15  - date1: 2018-03-01 00:00:00  - date0: 2018-02-14 00:00:00  - ARI: 0.9824309071024684
------------c1 new: [1 1 1 ... 3 3 3]
------------c1 init: [0 0 0 ... 2 2 2]
model: 5  - n_days: 15  - date1: 2018-03-16 00:00:00  - date0: 2018-03-01 00:00:00  - ARI: 0.9996053493346782
------------c1 new: [1 1 1 ... 2 2 3]
------------c1 init: [1 1 

C:\Users\aurel\AppData\Local\Temp/ipykernel_14552/2466628106.py:28: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  kmeans_m1.fit(file1_std1)


------------c1 new: [2 1 1 0 0]
------------c1 init: [4 2 2 2 2]
model: 18  - n_days: 15  - date1: 2018-09-27 00:00:00  - date0: 2018-09-12 00:00:00  - ARI: 0.2857142857142857
model: 19  - n_days: 15  - date1: 2018-10-12 00:00:00  - date0: 2018-09-27 00:00:00  - ARI: 0
model: 20  - n_days: 15  - date1: 2018-10-27 00:00:00  - date0: 2018-10-12 00:00:00  - ARI: 0


In [66]:
result_ARI =  pd.DataFrame(metrics_history)

In [67]:
result_ARI

,model,n_days,date1,date0,ARI
0,1,15,2018-01-15,2017-12-31,0.977210
1,2,15,2018-01-30,2018-01-15,0.980318
2,3,15,2018-02-14,2018-01-30,0.231714
3,4,15,2018-03-01,2018-02-14,0.982431
4,5,15,2018-03-16,2018-03-01,0.999605
5,6,15,2018-03-31,2018-03-16,0.978141
6,7,15,2018-04-15,2018-03-31,0.972276
7,8,15,2018-04-30,2018-04-15,0.975465
8,9,15,2018-05-15,2018-04-30,0.999593
9,10,15,2018-05-30,2018-05-15,0.427382


## Analyse des résultats obtenus

In [72]:
import plotly.express as px
fig=px.line(result_ARI, x="date0", y="ARI", title='ARI par date d\'achat (max par client)')
fig.update_layout(
plot_bgcolor="white")
fig.data[0].line.color = '#0C29D0'
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='grey', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='grey', zeroline=False)
fig.show()

Nous observons une baisse de l'ARI tous les 3 mois. Il nous faut donc mettre à jour notre modèle tous les 3 mois.